In [1]:
import pandas as pd
import numpy as np
import os
import pyomo.environ as en
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition
from pyomo.util.infeasible import log_infeasible_constraints
from Electrolyzer_v11b import build_model, GetStaticOutputs
import pickle
import gzip
import copy
import logging
import time

# Run using the TEMOA Environment, but with `pyomo=6.0`

logging.basicConfig(filename='direct_run_b.log', level=logging.INFO)

In [2]:
st = time.time()

In [3]:
cost_scenario = '2020_AG'
cost_scenario_folder = os.path.join('CostScenarios', cost_scenario)

StorageData = pd.read_excel(os.path.join(cost_scenario_folder, 'StorageData.xlsx'),index_col=[0])
Discount_rate = 0.054  # Discount rate %
Lifetime = 20.0  # Lifetime in years
# Capital charge factor to annualize investment costs
CCF_val = 1/float((Discount_rate+1)/float(Discount_rate)*(1-1/(1+Discount_rate)**Lifetime))  
PVData = pd.read_excel(os.path.join(cost_scenario_folder, 'PVData.xlsx'),'Data',index_col=[0]) # PV cost data
ElyData = pd.read_excel(os.path.join(cost_scenario_folder, 'ElyData.xlsx'),'Data',index_col=[0]) # Electrolyzer cost data
H2StData = pd.read_excel(os.path.join(cost_scenario_folder, 'H2StData.xlsx'),'Data',index_col=[0]) # H2 storage cost data
cf_file = '5796_23.65_68.75_tmy.csv' # PV resource availability defined for a single location

#PVAvail_tmy = read_PV_avail_df(cf_file)

PVAvail_tmy = pd.read_csv(cf_file,
                          index_col=0, 
                          parse_dates=True,
                          header=None,
                          squeeze=True)

productionCommitmentLB = int(np.floor(len(PVAvail_tmy) * .95)) # Minimum requirement for annual plant availability =95%
minimumProductionShutdownLength = 12 # Minimum number of hours system has to be turned down
P_Electricity = 120.0 #$/MWh # Price of exported grid electricity
LMPData = pd.Series(P_Electricity, index=range(len(PVAvail_tmy))) # 8760 x 1 vector of electricity prices 

In [4]:
m = build_model(pvavailarray=PVAvail_tmy.values,
                pricearray=LMPData.values,
                dfPVData=PVData,
                dfStorData=StorageData,
                dfElyData=ElyData,
                dfH2StData=H2StData,
                CCF=CCF_val,
                productionCommitmentLB=productionCommitmentLB,
                minimumProductionShutdownLength=minimumProductionShutdownLength)

[    0.27] Resetting the tic/toc delta timer
[+   0.25] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 231 in build_model
defined parameters 0
[+   0.00] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 237 in build_model
defined PV fixcost 0
[+   0.00] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 260 in build_model
defined H2 fixcost 0
[+   0.13] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 270 in build_model
defined sys variable cost 0
[+   0.04] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 293 in build_model
defined objective function 0
[+   0.21] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 307 in build_model
defined PV constraints 0
[+   0.67] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyzer_v11b.py", line 345 in build_model
defined storage constraints 0
[+   0.88] File "/Users/gta/Desktop/mallapragada-2020b/Electrolyze

In [5]:
opt = SolverFactory('cplex')
opt.options['mipgap'] = 0.01
opt.options['timelimit'] = 60*60*1
results = opt.solve(m, tee = True)
log_infeasible_constraints(m, log_expression=True, log_variables=True)


Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 12.9.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2019.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile '/var/folders/q5/ny1flrzs6nv9nk5k366w2gtr0000gn/T/tmpm4ql8x96.cplex.log' open.
CPLEX> New value for mixed integer optimality gap tolerance: 0.01
CPLEX> New value for time limit in seconds: 3600
CPLEX> Problem '/var/folders/q5/ny1flrzs6nv9nk5k366w2gtr0000gn/T/tmpjonc7ucp.pyomo.lp' read.
Read time = 0.86 sec. (26.47 ticks)
CPLEX> Problem name         : /var/folders/q5/ny1flrzs6nv9nk5k366w2gtr0000gn/T/tmpjonc7ucp.pyomo.lp
Objective sense      : Minimize
Variables            :  131423  [Nneg: 122662,  Binary: 8761]
Objective nonzeros   :   43812
Linear constraints   :  262833  [Less: 192743,  Great

In [6]:
if (results.solver.status == SolverStatus.ok):
    print('Normal termination')

elif (results.solver.status == SolverStatus.warning):
    print('Termination with unusual condition')

elif (results.solver.status == SolverStatus.error):
    print('Terminated internally with error')

elif (results.solver.status == SolverStatus.aborted):
    print('Terminated due to external conditions (e.g. interrupts)')

elif (results.solver.status == SolverStatus.unknown):
    print('Unknown (an uninitialized value)')
else:
    print("Solver Status: {}".format(results.solver.status))

Normal termination


In [7]:
if (results.solver.termination_condition == TerminationCondition.optimal):
    print('Optimal Solution Found')
elif (results.solver.termination_condition == TerminationCondition.maxTimeLimit):
    print('Exceeded maximum time limit allowed')

elif (results.solver.termination_condition == TerminationCondition.maxIterations):
    print('Exceeded maximum number of iterations allowed')

elif (results.solver.termination_condition == TerminationCondition.minFunctionValue):
    print('Found solution smaller than specified function value')

elif (results.solver.termination_condition == TerminationCondition.minStepLength):
    print('Step length is smaller than specified limit')

elif (results.solver.termination_condition == TerminationCondition.globallyOptimal):
    print('Found a globally optimal solution')

elif (results.solver.termination_condition == TerminationCondition.locallyOptimal):
    print('Found a locally optimal solution')

elif (results.solver.termination_condition == TerminationCondition.optimal):
    print('Found an optimal solution')

elif (results.solver.termination_condition == TerminationCondition.maxEvaluations):
    print('Exceeded maximum number of problem evaluations (e.g., branch and bound nodes)')

elif (results.solver.termination_condition == TerminationCondition.other):
    print('Other, uncategorized normal termination')

elif (results.solver.termination_condition == TerminationCondition.unbounded):
    print('Demonstrated that problem is unbounded')

elif (results.solver.termination_condition == TerminationCondition.infeasible):
    print('Demonstrated that problem is infeasible')

elif (results.solver.termination_condition == TerminationCondition.invalidProblem):
    print('The problem setup or characteristics are not valid for the solver')

elif (results.solver.termination_condition == TerminationCondition.solverFailure):
    print('Solver failed to terminate correctly')

elif (results.solver.termination_condition == TerminationCondition.internalSolverError):
    print('Internal solver error')

elif (results.solver.termination_condition == TerminationCondition.error):
    print('Other error')

elif (results.solver.termination_condition == TerminationCondition.userInterrupt):
    print('Interrupt signal generated by user')

elif (results.solver.termination_condition == TerminationCondition.resourceInterrupt):
    print('Interrupt signal in resources used by the solver')

elif (results.solver.termination_condition == TerminationCondition.licensingProblem):
    print('Problem accessing solver license')
else:
    print("Solver Status: {}".format(results.solver.status))

Demonstrated that problem is unbounded


In [8]:
et = time.time()

In [9]:
delta = (et - st) / 60
print('Elapsed Time: {:.3f} min'.format(delta))

Elapsed Time: 248.248 min


In [12]:
en.value(m.pH2kgpertank)

1000

In [13]:
en.value(m.vPVInstalledMW)

ValueError: No value for uninitialized NumericValue object vPVInstalledMW

In [14]:
en.value(m.vPVtoDCPow)

TypeError: 'IndexedVar' object is not callable

In [15]:
en.value(m.vPVtoDCPow[10])

ValueError: No value for uninitialized NumericValue object vPVtoDCPow[10]

In [18]:
en.value(m.vCompInstalledMW)

ValueError: No value for uninitialized NumericValue object vCompInstalledMW

In [22]:
en.value(m.pCapCostH2st)

516

In [23]:
en.value(m.pFOMCostH2st)

5.16

In [24]:
en.value(m.pCompSpecPower)

1